# Pyspark_learning_notes

In [39]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [40]:
sc.master

'local[*]'

## textFile读取本地文件和hdfs文件

In [41]:
textfile=sc.textFile("file:/c:/f/test.txt")#本地文件注意路径书写格式file:

In [42]:
textfile.collect()

['300', '200', '22']

In [43]:
textfile.count()

3

In [44]:
textfile=sc.textFile("hdfs:/hdfsfile/test/test.txt")#hdfs文件注意路径书写格式hdfs:

In [45]:
textfile.collect()

['300', '200', '22']

## RDD的转换及运算

In [46]:
intRDD=sc.parallelize(list((3,1,2,5,5)))#创建RDD

In [67]:
intRDD

ParallelCollectionRDD[38] at parallelize at PythonRDD.scala:175

In [47]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [48]:
strRDD=sc.parallelize(list(('app','note','oran','app','grap')))

In [49]:
strRDD.collect()

['app', 'note', 'oran', 'app', 'grap']

### map的用法

In [50]:
intRDD.map(lambda x:x+1).collect()

[4, 2, 3, 6, 6]

In [51]:
strRDD.map(lambda x:'ncol_'+x).collect()

['ncol_app', 'ncol_note', 'ncol_oran', 'ncol_app', 'ncol_grap']

### filter筛选

In [52]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [53]:
intRDD.filter(lambda x:x>=3).collect()

[3, 5, 5]

In [54]:
intRDD.filter(lambda x:1<x<=3).collect()

[3, 2]

In [55]:
strRDD.filter(lambda x: 'ap' in x).collect()

['app', 'app', 'grap']

### distinct删除重复项

In [56]:
intRDD.distinct().collect()

[1, 5, 2, 3]

In [57]:
strRDD.distinct().collect()

['app', 'note', 'oran', 'grap']

### randomSplit将整个集合的元素以随机数的方式按照比例分为多个RDD

In [58]:
strRDD.collect()

['app', 'note', 'oran', 'app', 'grap']

In [59]:
sRDD=strRDD.randomSplit([0.4,0.6])

In [60]:
sRDD[0].collect()

['app', 'note']

In [61]:
sRDD[1].collect()

['oran', 'app', 'grap']

### groupBy聚合运算

In [62]:
gRDD=intRDD.groupBy(lambda x:'even' if (x%2==0) else 'odd').collect()#按奇偶分离

In [63]:
gRDD

[('even', <pyspark.resultiterable.ResultIterable at 0x28d5ee23d68>),
 ('odd', <pyspark.resultiterable.ResultIterable at 0x28d5ee23e10>)]

In [64]:
gRDD[0][0],sorted(gRDD[0][1])#这里的sorted方法也可用list、tuple方法

('even', [2])

In [65]:
gRDD[1][0],sorted(gRDD[1][1])

('odd', [1, 3, 5, 5])

### union合并运算

In [68]:
intRDD1=sc.parallelize([3,1,2,5,5])
intRDD2=sc.parallelize([5,6])
intRDD3=sc.parallelize([2,7])

In [69]:
intRDD1.union(intRDD2).collect()

[3, 1, 2, 5, 5, 5, 6]

In [70]:
intRDD1.union(intRDD2).union(intRDD3).collect()

[3, 1, 2, 5, 5, 5, 6, 2, 7]

### intersection交集运算

In [71]:
intRDD1.intersection(intRDD2).collect()

[5]

### subtract差集运算

In [72]:
intRDD1.subtract(intRDD2).collect()

[1, 2, 3]

### cartesion笛卡尔合并

In [73]:
intRDD2.cartesian(intRDD3).collect()

[(5, 2), (5, 7), (6, 2), (6, 7)]

## RDD基本“动作”运算

In [76]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [77]:
intRDD.first()

3

In [78]:
intRDD.take(2)#取前两项

[3, 1]

In [79]:
intRDD.takeOrdered(3)#取前三项并排序

[1, 2, 3]

In [80]:
intRDD.takeOrdered(3,key=lambda x:-x)#从大到小排序后取前三项

[5, 5, 3]

In [88]:
intRDD.takeSample(True,10,123)#参数True表示放回样本采样，10表示采样个数，123表示随机种子

[2, 5, 5, 5, 2, 1, 1, 2, 2, 1]

In [89]:
intRDD.takeSample(False,10,123)

[5, 1, 5, 2, 3]

### RDD统计运算

In [91]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [90]:
intRDD.stats()

(count: 5, mean: 3.2, stdev: 1.6, max: 5.0, min: 1.0)

In [92]:
intRDD.stdev()#标准差，

1.6

In [93]:
intRDD.sum()

16

## RDD的key-value基本转换 

In [105]:
kvRDD=sc.parallelize([(1,2),(3,4),(5,6),(5,7)])#创建RDD key-value实例

In [106]:
kvRDD.collect()

[(1, 2), (3, 4), (5, 6), (5, 7)]

In [107]:
kvRDD.keys().collect()

[1, 3, 5, 5]

In [108]:
kvRDD.values().collect()

[2, 4, 6, 7]

In [109]:
kvRDD.filter(lambda kv:kv[0]<4).collect()#筛选出key值小于4的元素

[(1, 2), (3, 4)]

In [110]:
kvRDD.filter(lambda kv:kv[1]<4).collect()#筛选出value值小于4的元素

[(1, 2)]

### mapValues针对RDD内的每一组(key,value)的value进行运算，并且产生新的RDD

In [111]:
kvRDD.mapValues(lambda x:x*2).collect()

[(1, 4), (3, 8), (5, 12), (5, 14)]